In [6]:
import hashlib

### test

In [24]:
Pcurve = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1

In [55]:
#convert ehxa to decimal
def hex_to_decimal(hex_str):
    return int(hex_str, 16)

hex_to_decimal("ffffffff00000001000000000000000000000000ffffffffffffffffffffffff")

115792089210356248762697446949407573530086143415290314195533631308867097853951

In [56]:
Pcurve

115792089237316195423570985008687907853269984665640564039457584007908834671663

In [70]:
import hashlib
from hmac import HMAC
from hashlib import sha256

Pcurve = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff  # The proven prime
N = 0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 # Number of points in the field
Acurve = 0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc
Bcurve = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b  # This defines the curve. y^2 = x^3 + Acurve * x + Bcurve
Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5
GPoint = (Gx, Gy)  # This is our generator point. Tillions of dif ones possible

# Individual Transaction/Personal Information
privKey = 0xC9AFA9D845BA75166B5C215767B1D6934E50C3DB36E89B127B8A622B120F6721  # replace with any private key
#RandNum = 28695618543805844332113829720373285210420739438570883203839696518176414791234  # replace with a truly random number
message = 'sample'
HashOfThingToSign = int(hashlib.sha256(message.encode()).hexdigest(), 16)



def modinv(a, n=Pcurve):  # Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1, 0
    low, high = a % n, n
    while low > 1:
        ratio = high // low  # Use integer division operator for Python 3.x
        nm, new = hm - lm * ratio, high - low * ratio
        lm, low, hm, high = nm, new, lm, low
    return lm % n


def ECadd(xp, yp, xq, yq):  # Not true addition, invented for EC. It adds Point-P with Point-Q.
    m = ((yq - yp) * modinv(xq - xp, Pcurve)) % Pcurve
    xr = (m * m - xp - xq) % Pcurve
    yr = (m * (xp - xr) - yp) % Pcurve
    return (xr, yr)


def ECdouble(xp, yp):  # EC point doubling, invented for EC. It doubles Point-P.
    LamNumer = 3 * xp * xp + Acurve
    LamDenom = 2 * yp
    Lam = (LamNumer * modinv(LamDenom, Pcurve)) % Pcurve
    xr = (Lam * Lam - 2 * xp) % Pcurve
    yr = (Lam * (xp - xr) - yp) % Pcurve
    return (xr, yr)


def EccMultiply(xs, ys, Scalar):  # Double & add. EC Multiplication, Not true multiplication
    if Scalar == 0 or Scalar >= N:
        raise Exception("Invalid Scalar/Private Key")
    ScalarBin = str(bin(Scalar))[2:]
    Qx, Qy = xs, ys
    for i in range(1, len(ScalarBin)):  # This is invented EC multiplication.
        Qx, Qy = ECdouble(Qx, Qy)
        if ScalarBin[i] == "1":
            Qx, Qy = ECadd(Qx, Qy, xs, ys)
    return (Qx, Qy)

def deterministic_generate_k(msghash, privkey):
    # Step 1: Prepare the key and message
    V = b'\x01' * 32
    K = b'\x00' * 32
    privkey = privkey.to_bytes(32, 'big')
    msghash = msghash.to_bytes(32, 'big')

    # Step 2: Generate the K value
    K = HMAC(K, V + b'\x00' + privkey + msghash, sha256).digest()
    V = HMAC(K, V, sha256).digest()
    K = HMAC(K, V + b'\x01' + privkey + msghash, sha256).digest()
    V = HMAC(K, V, sha256).digest()

    # Step 3: Convert the K value to an integer
    while True:
        t = b''
        while len(t) < 32:
            V = HMAC(K, V, sha256).digest()
            t += V
        k = int.from_bytes(t[:32], 'big')
        if k >= 1 and k < N:
            return k

# ...

def ECDSA_sign(message, privKey):
    # Step 1: Generate the deterministic k value
    HashOfThingToSign = int(hashlib.sha256(message.encode()).hexdigest(), 16)
    k = deterministic_generate_k(HashOfThingToSign, privKey)

    # Step 2: Compute the public key
    xPublicKey, yPublicKey = EccMultiply(Gx, Gy, privKey)

    # Step 3: Compute r
    xRandSignPoint, yRandSignPoint = EccMultiply(Gx, Gy, k)
    r = xRandSignPoint % N

    # Step 4: Compute s
    s = ((HashOfThingToSign + r * privKey) * (modinv(k, N))) % N

    return r, s

def ECDSA_verify(message, r, s, xPublicKey, yPublicKey):
    # Step 1: Compute the deterministic k value
    HashOfThingToSign = int(hashlib.sha256(message.encode()).hexdigest(), 16)
    k = deterministic_generate_k(HashOfThingToSign, xPublicKey)

    # Step 2: Compute w
    w = modinv(s, N)

    # Step 3: Compute u1 and u2
    xu1, yu1 = EccMultiply(Gx, Gy, (HashOfThingToSign * w) % N)
    xu2, yu2 = EccMultiply(xPublicKey, yPublicKey, (r * w) % N)

    # Step 4: Compute the verification point (x, y)
    x, y = ECadd(xu1, yu1, xu2, yu2)

    # Step 5: Check if r equals x
    return r == x


# Step 1: Generate the deterministic k value
k = deterministic_generate_k(HashOfThingToSign, privKey)

# Step 2: Compute the public key
xPublicKey, yPublicKey = EccMultiply(Gx, Gy, privKey)

# Step 3: Compute r
xRandSignPoint, yRandSignPoint = EccMultiply(Gx, Gy, k)
r = xRandSignPoint % N

# Step 4: Compute s
s = ((HashOfThingToSign + r * privKey) * (modinv(k, N))) % N

# ...

print("******* Public Key Generation *********")
xPublicKey, yPublicKey = EccMultiply(Gx, Gy, privKey)
print("Private key (hex):", hex(k))
print("Uncompressed public key (hex):", "04" + format(xPublicKey, '064x') + format(yPublicKey, '064x'))

print("\n******* Signature Generation *********")
r, s = ECDSA_sign(message, privKey)
print("r (hex):", hex(r))
print("s (hex):", hex(s))

print("\n******* Signature Verification *********")
valid_signature = ECDSA_verify(message, r, s, xPublicKey, yPublicKey)
print("Valid signature:", valid_signature)

print("\n******* Public keys *********")
print("xPublicKey:", hex(xPublicKey))
print("yPublicKey:", hex(yPublicKey))

******* Public Key Generation *********
Private key (hex): 0xa6e3c57dd01abe90086538398355dd4c3b17aa873382b0f24d6129493d8aad60
Uncompressed public key (hex): 0460fed4ba255a9d31c961eb74c6356d68c049b8923b61fa6ce669622e60f29fb67903fe1008b8bc99a41ae9e95628bc64f2f1b20c2d7e9f5177a3c294d4462299

******* Signature Generation *********
r (hex): 0xefd48b2aacb6a8fd1140dd9cd45e81d69d2c877b56aaf991c34d0ea84eaf3716
s (hex): 0xf7cb1c942d657c41d436c7a1b6e29f65f3e900dbb9aff4064dc4ab2f843acda8

******* Signature Verification *********
Valid signature: True

******* Public keys *********
xPublicKey: 0x60fed4ba255a9d31c961eb74c6356d68c049b8923b61fa6ce669622e60f29fb6
yPublicKey: 0x7903fe1008b8bc99a41ae9e95628bc64f2f1b20c2d7e9f5177a3c294d4462299


In [63]:
r1 = 'efd48b2aacb6a8fd1140dd9cd45e81d69d2c877b56aaf991c34d0ea84eaf3716'
r2 = 'EFD48B2AACB6A8FD1140DD9CD45E81D69D2C877B56AAF991C34D0EA84EAF3716'

print(r1.lower() == r2.lower())

True


In [66]:
s1 = 'f7cb1c942d657c41d436c7a1b6e29f65f3e900dbb9aff4064dc4ab2f843acda8'
s2 = 'F7CB1C942D657C41D436C7A1B6E29F65F3E900DBB9AFF4064DC4AB2F843ACDA8'

print(s1.lower() == s2.lower())

True


In [68]:
k1 = 'a6e3c57dd01abe90086538398355dd4c3b17aa873382b0f24d6129493d8aad60'
k2 = 'A6E3C57DD01ABE90086538398355DD4C3B17AA873382B0F24D6129493D8AAD60'

print(k1.lower() == k2.lower())

True


- With SHA-256, message = "sample":
- k = A6E3C57DD01ABE90086538398355DD4C3B17AA873382B0F24D6129493D8AAD60
- r = EFD48B2AACB6A8FD1140DD9CD45E81D69D2C877B56AAF991C34D0EA84EAF3716
- s = F7CB1C942D657C41D436C7A1B6E29F65F3E900DBB9AFF4064DC4AB2F843ACDA8

## etc...

In [17]:
message = """{"trace_id": "10", "timestamp": "2013-11-02T02:15:00Z", "C(0)/P(1)": "0", "value": 0.06}"""


In [18]:
int(hashlib.sha256(message.encode()).hexdigest(), 16)

49461256774148754317813947867449981477932251401967141759450982441322678275816

In [19]:
message.encode()

b'{"trace_id": "10", "timestamp": "2013-11-02T02:15:00Z", "C(0)/P(1)": "0", "value": 0.06}'

In [22]:
hashlib.sha256(message.encode()).hexdigest()

'6d5a144a7959aac9e3602aff8c7c2ab14298c8344366efd70e3b7d0f120f4ee8'

In [23]:
int(hashlib.sha256(message.encode()).hexdigest(), 16)

49461256774148754317813947867449981477932251401967141759450982441322678275816

### implementation test

In [52]:
# Python 3.x - Super simple Elliptic Curve Presentation. No imported libraries, wrappers, nothing. # For educational purposes only
# Below are the public specs for Bitcoin's curve - the secp256k1
Pcurve = 2**256 - 2**32 - 2**9 - 2**8 - 2**7 - 2**6 - 2**4 - 1  # The proven prime
N = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141  # Number of points in the field
Acurve = 0
Bcurve = 7  # This defines the curve. y^2 = x^3 + Acurve * x + Bcurve
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
GPoint = (Gx, Gy)  # This is our generator point. Tillions of dif ones possible

# Individual Transaction/Personal Information
privKey = 75263518707598184987916378021939673586055614731957507592904438851787542395619  # replace with any private key
RandNum = 28695618543805844332113829720373285210420739438570883203839696518176414791234  # replace with a truly random number
message = "sample"
HashOfThingToSign = int(hashlib.sha256(message.encode()).hexdigest(), 16)



def modinv(a, n=Pcurve):  # Extended Euclidean Algorithm/'division' in elliptic curves
    lm, hm = 1, 0
    low, high = a % n, n
    while low > 1:
        ratio = high // low  # Use integer division operator for Python 3.x
        nm, new = hm - lm * ratio, high - low * ratio
        lm, low, hm, high = nm, new, lm, low
    return lm % n


def ECadd(xp, yp, xq, yq):  # Not true addition, invented for EC. It adds Point-P with Point-Q.
    m = ((yq - yp) * modinv(xq - xp, Pcurve)) % Pcurve
    xr = (m * m - xp - xq) % Pcurve
    yr = (m * (xp - xr) - yp) % Pcurve
    return (xr, yr)


def ECdouble(xp, yp):  # EC point doubling, invented for EC. It doubles Point-P.
    LamNumer = 3 * xp * xp + Acurve
    LamDenom = 2 * yp
    Lam = (LamNumer * modinv(LamDenom, Pcurve)) % Pcurve
    xr = (Lam * Lam - 2 * xp) % Pcurve
    yr = (Lam * (xp - xr) - yp) % Pcurve
    return (xr, yr)


def EccMultiply(xs, ys, Scalar):  # Double & add. EC Multiplication, Not true multiplication
    if Scalar == 0 or Scalar >= N:
        raise Exception("Invalid Scalar/Private Key")
    ScalarBin = str(bin(Scalar))[2:]
    Qx, Qy = xs, ys
    for i in range(1, len(ScalarBin)):  # This is invented EC multiplication.
        Qx, Qy = ECdouble(Qx, Qy)
        if ScalarBin[i] == "1":
            Qx, Qy = ECadd(Qx, Qy, xs, ys)
    return (Qx, Qy)


print("\n******* Public Key Generation *********")
xPublicKey, yPublicKey = EccMultiply(Gx, Gy, privKey)
print("the private key (in base 10 format):")
print(privKey)
print("\nthe uncompressed public key (starts with '04' & is not the public address):")
print("04", xPublicKey, yPublicKey)

print("\n******* Signature Generation *********")
xRandSignPoint, yRandSignPoint = EccMultiply(Gx, Gy, RandNum)
r = xRandSignPoint % N
print("r =", r)
s = ((HashOfThingToSign + r * privKey) * (modinv(RandNum, N))) % N
print("s =", s)

print("\n******* Signature Verification *********")
w = modinv(s, N)
xu1, yu1 = EccMultiply(Gx, Gy, (HashOfThingToSign * w) % N)
xu2, yu2 = EccMultiply(xPublicKey, yPublicKey, (r * w) % N)
x, y = ECadd(xu1, yu1, xu2, yu2)
print(r == x)



******* Public Key Generation *********
the private key (in base 10 format):
75263518707598184987916378021939673586055614731957507592904438851787542395619

the uncompressed public key (starts with '04' & is not the public address):
04 86123958339353589454334613954037009250298301442165544159467110006827437489844 24886167583395101331704142008829378675881745768490090403994652035580982362733

******* Signature Generation *********
r = 108450790312736419148091503336190989867379581793003243037811027177541631669413
s = 10733996259657884841413429023141444606165984322107338968372818586196368897459

******* Signature Verification *********
True
